### GPU

人们使用GPU来进行机器学习，因为单个GPU相对运行速度快。**但是在设备（CPU、GPU和其他机器）之间传输数据比计算慢得多。这也使得并行化变得更加困难**，因为我们必须等待数据被发送（或者接收），然后才能继续进行更多的操作。**这就是为什么拷贝操作要格外小心。根据经验，多个小操作比一个大操作糟糕得多。此外，一次执行几个操作比代码中散布的许多单个操作要好得多。如果一个设备必须等待另一个设备才能执行其他操作，那么这样的操作可能会阻塞**。这有点像排队订购咖啡，而不像通过电话预先订购：当客人到店的时候，咖啡已经准备好了。

**最后，当我们打印张量或将张量转换为NumPy格式时，如果数据不在内存中，框架会首先将其复制到内存中，这会导致额外的传输开销。更糟糕的是，它现在受制于全局解释器锁，使得一切都得等待Python完成**。

- 首先，确保至少安装了一个 NVIDIA GPU

- 然后，下载 NVIDIA 驱动程序和 CUDA

- 最后，通过 `nvidia-smi` 命令来查看显卡信息

In [7]:
!nvidia-smi

Sat Aug 12 19:17:19 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.54       Driver Version: 526.56       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   48C    P8     4W /  N/A |    788MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**计算设备**

- **在 PyTorch 中，CPU 和 GPU 可以用 `torch.device('cpu')` 和 `torch.device('cuda')` 表示**

- 应该注意的是，CPU 设备代表所有物理 CPU 和内存，这意味着 PyTorch 的计算将尝试使用所有 CPU 核

- 然而，GPU 设备只代表一个卡和相应的显存

- 如果有多个 GPU，我们使用 `torch.device(f'cuda:{i}')` 来表示第 i 个 GPU（这里 i 从 0 开始）

- **`cuda:0` 和 `cuda` 是等价的**

In [8]:
import torch
from torch import nn

torch.device("cpu"), torch.cuda.device("cuda"), torch.cuda.device("cuda:1")

(device(type='cpu'),
 <torch.cuda.device at 0x7f0858ff3af0>)

**查询可用GPU的数量**

In [9]:
torch.cuda.device_count()

1

**这两个函数允许我们在请求的GPU不存在的情况下运行代码**

In [10]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f"cuda:{i}")
    return torch.device("cpu")


def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f"cuda:{i}") for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

**查询张量所在的设备**

- **需要注意的是，无论何时我们要对多个项进行操作，它们都必须在同一个设备上**

In [11]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

**存储在GPU上**

- **在 GPU 上创建的张量只消耗这个 GPU 的显存，我们可以使用 `nvidia-smi` 命令查看显存的使用情况**

In [12]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

**第二个GPU上创建一个随机张量**

In [14]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.1810, 0.3659, 0.8204],
        [0.9376, 0.1340, 0.5462]], device='cuda:0')

**要计算`X + Y`，我们需要决定在哪里执行这个操作**

In [16]:
Z = X.cuda(1)   # 将张量X从GPU 0 copy到GPU 1
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


**现在数据都在同一个GPU上（`Z` 和 `Y` 都在），我们可以将它们相加**

如果数据在不同的`device`，来回挪动东西就会很麻烦，故而pytorch直接报错，不允许不同设备的数据做运算

In [17]:
Y + Z

tensor([[1.1810, 1.3659, 1.8204],
        [1.9376, 1.1340, 1.5462]], device='cuda:0')

**检查一个张量 `Z` 是否已经被移动到 GPU 设备上的特定 GPU 设备编号（GPU 0）**

**检查原始张量 Z 是否与经过 .cuda(0) 移动到 GPU 0 后的张量相同。请注意，这里的检查只会考虑张量的身份（内存地址），而不会考虑内容是否相同。**

In [18]:
Z.cuda(0) is Z  # Z.cuda(0)返回的是Z本身

True

--------------------------

**神经网络与GPU**

In [19]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([[-0.1669],
        [-0.1669]], device='cuda:0', grad_fn=<AddmmBackward0>)

**确认模型参数存储在同一个GPU上**

In [20]:
net[0].weight.data.device, X.device

(device(type='cuda', index=0), device(type='cuda', index=0))

> 总之，只要所有的数据和参数都在同一个设备上，我们就可以有效地学习模型

**不经意地移动数据可能会显著降低性能**。一个典型的错误如下：计算GPU上每个小批量的损失，并在命令行中将其报告给用户（或将其记录在NumPy `ndarray`中）时，**将触发全局解释器锁，从而使所有GPU阻塞**。最好是为GPU内部的日志分配内存，并且只移动较大的日志。